In [1]:
%matplotlib inline
import os
import sys
import healpy as hp
import numpy as np
import matplotlib.pyplot as plt
import pickle as pk
sys.path.append('../')
import matplotlib

In [2]:
%load_ext autoreload
%autoreload 2
import libparam as method

usage: ipykernel_launcher.py [-h] [-ivt] [-ivp] [-dd] [-delens] [-cl] [-lh]
                             inifile
ipykernel_launcher.py: error: unrecognized arguments: -f


Simulation uses /global/cscratch1/sd/lonappan/S4BIRD/LiteBIRD/SIM_SET1/Maps
jit: instantiating ctype = <class 'plancklens.qcinv.multigrid.multigrid_chain'>
creating split preconditioner  ('dense(/global/cscratch1/sd/lonappan/S4BIRD/LiteBIRD/SIM_SET1/QE/cinv_t/dense.pk)', '64', 'diag_cl')
creating dense preconditioner. (nside = 128, lmax = 64, cache = /global/cscratch1/sd/lonappan/S4BIRD/LiteBIRD/SIM_SET1/QE/cinv_t/dense.pk)
jit: instantiating ctype = <class 'plancklens.qcinv.opfilt_tt.alm_filter_ninv'>
opfilt_tt: inverse noise map std dev / av = 5.164e-14
 [00:00:00] filling template (4) projection matrix > 00%

 [00:00:07] filling template (4) projection matrix -------> 75%
ninv_ftl: using 2.15 uK-amin noise Cl
DEGRADING WITH NO MARGE MAPS
opfilt_tt: inverse noise map std dev / av = 1.710e-16
 [00:00:00] filling template (4) projection matrix -------> 75%
ninv_ftl: using 2.15 uK-amin noise Cl
computing dense preconditioner:
     lmax  = 64
     ntmpl = 4
 will cache minv in /global/cscratch1/sd/lonappan/S4BIRD/LiteBIRD/SIM_SET1/QE/cinv_t/dense.pk


 [00:00:50] filling matrix ---------> 99%
   inverting M...
     eigv[ntmpl-1] =  4.671918243294272e-10
     eigv[ntmpl]   =  0.0030318083730572223
creating split preconditioner  ('stage(3)', '256', 'diag_cl')
creating multigrid preconditioner: stage_id =  3
DEGRADING WITH NO MARGE MAPS
opfilt_tt: inverse noise map std dev / av = 1.710e-16
 [00:00:00] filling template (4) projection matrix -------> 75%
ninv_ftl: using 2.15 uK-amin noise Cl
creating split preconditioner  ('stage(2)', '512', 'diag_cl')
creating multigrid preconditioner: stage_id =  2
DEGRADING WITH NO MARGE MAPS
opfilt_tt: inverse noise map std dev / av = 8.550e-16
 [00:00:00] filling template (4) projection matrix -------> 75%
ninv_ftl: using 2.15 uK-amin noise Cl
creating split preconditioner  ('stage(1)', '1024', 'diag_cl')
creating multigrid preconditioner: stage_id =  1
DEGRADING WITH NO MARGE MAPS
opfilt_tt: inverse noise map std dev / av = 2.052e-15
 [00:00:00] filling template (4) projection matrix -------> 75%
n

setting the output map dtype to [dtype('float64')]


jit: instantiating ctype = <class 'plancklens.qcinv.multigrid.multigrid_chain'>
creating split preconditioner  ('dense(/global/cscratch1/sd/lonappan/S4BIRD/LiteBIRD/SIM_SET1/QE/cinv_p/dense.pk)', '32', 'diag_cl')
creating dense preconditioner. (nside = 256, lmax = 32, cache = /global/cscratch1/sd/lonappan/S4BIRD/LiteBIRD/SIM_SET1/QE/cinv_p/dense.pk)
jit: instantiating ctype = <class 'plancklens.qcinv.opfilt_pp.alm_filter_ninv'>
opfilt_pp: Pol. inverse noise map std dev / av = 3.212e-14
ninv_febl: using 3.06 uK-amin noise Cl
opfilt_pp: Pol. inverse noise map std dev / av = 5.180e-16
ninv_febl: using 3.06 uK-amin noise Cl
computing dense preconditioner:
     lmax  = 32
     ntmpl = 8
 [00:00:57] filling matrix ---------> 99%
   inverting M...
     eigv[ntmpl-1] =  1.4737125352165832e-10
     eigv[ntmpl]   =  808.8102138316367
creating split preconditioner  ('stage(2)', '512', 'diag_cl')
creating multigrid preconditioner: stage_id =  2
opfilt_pp: Pol. inverse noise map std dev / av = 5.18

setting the output map dtype to [dtype('float64')]
setting the output map dtype to [dtype('>f8')]


Caching sky fractions...
Delensing uses QE: p_eb


No workspace file found.
Computing Mode Coupling Matrix


In [3]:
method.sims.get_sim_elm(0)

array([ 0.00000000e+00+0.00000000e+00j,  0.00000000e+00+0.00000000e+00j,
        1.22713168e-01+0.00000000e+00j, ...,
        2.73115856e-19-4.02457938e-19j,  7.59975687e-19+1.99214919e-19j,
       -1.37668320e-19-1.17490762e-18j])